# Import relevant libraries

In [1]:
import re
import os
import numpy as np
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

# Load data

In [2]:
# set base_path
base_path = os.path.join('..', '..')
os.chdir(base_path)
os.getcwd()

'C:\\Users\\G7\\Desktop\\Cass Study\\SMM635 Data Visualisation\\FTP'

In [3]:
# set file_path
file_path = os.path.join('datasets', 'cleaned_data.csv')

# load data
company_data = pd.read_csv(file_path, low_memory=False)

In [4]:
company_data.head()

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
0,! LIMITED,12778855,NaN,NaN,UNIT 3 NEWTON BUSINESS CENTRE,NEWTON CHAMBERS ROAD,SHEFFIELD,NaN,UNITED KINGDOM,S35 2PH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/08/2021,NaN
1,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2021,11/09/2020
2,!? LTD,11399177,NaN,NaN,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,UNITED KINGDOM,SK6 3DY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/06/2021,05/06/2020
3,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/02/2021,27/12/2019
4,!L PRODUCTIONS LIMITED,12402527,NaN,NaN,95 MARMION AVENUE,NaN,CHINGFORD,NaN,ENGLAND,E4 8EJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24/02/2021,NaN


In [5]:
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4785590 entries, 0 to 4785589
Data columns (total 55 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   CompanyName                         object 
 1   CompanyNumber                       object 
 2   RegAddress.CareOf                   object 
 3   RegAddress.POBox                    object 
 4   RegAddress.AddressLine1             object 
 5   RegAddress.AddressLine2             object 
 6   RegAddress.PostTown                 object 
 7   RegAddress.County                   object 
 8   RegAddress.Country                  object 
 9   RegAddress.PostCode                 object 
 10  CompanyCategory                     object 
 11  CompanyStatus                       object 
 12  CountryOfOrigin                     object 
 13  DissolutionDate                     float64
 14  IncorporationDate                   object 
 15  Accounts.AccountRefDay              float64
 16  

In [6]:
# prepare data
df = company_data[['CompanyName', 'RegAddress.PostCode', 'RegAddress.PostTown']]

# select only companies in London
df = df[df['RegAddress.PostTown'] == 'LONDON']

# remove na
df.dropna(inplace = True)

# select only East London
pattern = r'^EC[12]'

EC12 = []
for postcode in df['RegAddress.PostCode']:
    val = bool(re.match(pattern, postcode))
    EC12.append(val)

In [7]:
sum(EC12)

97969

In [ ]:
# get latitude, longitude
latitude = []
longitude = []

# request coordinate
geolocator = Nominatim(user_agent = "http")
for postcode in tqdm(df['RegAddress.PostCode']):
    location = geolocator.geocode(postcode)
    if location is None:
        latitude.append(None)
        longitude.append(None)
    if location is not None:
        latitude.append(location.latitude)
        longitude.append(location.longitude)